## **Machine Learning - WNBA Playoffs Prediction**
This notebook will focus on the preparation of the data. We will be using SQLite to store the data due to its scalability & the fact that it's a relational schema.

https://docs.python.org/3/library/sqlite3.html

Import sqlite3 and connect to database file

### **Imports**

In [8]:
import pandas as pd
import sqlite3
import prep_utils as pu 

### **Database Connection Setup**

In [9]:
import sqlite3
db = sqlite3.connect("db/ac.db")
db_cur = db.cursor()

**Descriptive Statistics**

Descriptive statistics offer valuable metrics for understanding and summarizing data, assisting in data analysis and decision-making.

In [10]:
tables = pu.get_db_tables(db_cur)

for table in tables:   
    print(f"\033[1m{table}\033[0m")
    num, non_num = pu.parse_columns_type(db_cur,table)
    pu.calculate_summary_statistics(db_cur,table,num)
    print('\n')


awards_players
+-------------+---------+---------+-----------------+-------+-------+
| Attribute   |   Count |    Mean |   Std Deviation |   Min |   Max |
+=============+=========+=========+=================+=======+=======+
| year        |      95 | 5.78947 |            2.75 |     1 |    10 |
+-------------+---------+---------+-----------------+-------+-------+


coaches
+-------------+---------+-----------+-----------------+-------+-------+
| Attribute   |   Count |      Mean |   Std Deviation |   Min |   Max |
+=============+=========+===========+=================+=======+=======+
| year        |     162 |  5.31481  |            2.9  |     1 |    10 |
+-------------+---------+-----------+-----------------+-------+-------+
| stint       |     162 |  0.364198 |            0.69 |     0 |     2 |
+-------------+---------+-----------+-----------------+-------+-------+
| won         |     162 | 14.6728   |            6.4  |     0 |    28 |
+-------------+---------+-----------+------------

Players:
- All players have the same value for first season & last season (0)
- Some players have height and weight with the value 0

Teams:
- The attributes 'seeded', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmDRB', 'opptmTRB' has the value 0 for all teams


**Checking for missing values (N/A)**

It's normal for large datasets to have missing values, which needs to be handled early on.

In [11]:
for table in tables:   
    print(f"\033[1m{table} - Missing Values:\033[0m")
    pu.check_missing_values(db_cur,table)
    print('\n')


awards_players - Missing Values:
There are no missing values!

coaches - Missing Values:
There are no missing values!

players - Missing Values:
Column 'pos' has missing values: True - 8.73%
Column 'college' has missing values: True - 18.70%
Column 'collegeOther' has missing values: True - 98.77%


teams - Missing Values:
Column 'divID' has missing values: True - 100.00%
Column 'firstRound' has missing values: True - 43.66%
Column 'semis' has missing values: True - 73.24%
Column 'finals' has missing values: True - 85.92%


teams_post - Missing Values:
There are no missing values!

series_post - Missing Values:
There are no missing values!

players_teams - Missing Values:
There are no missing values!

